# 04 Integration (P × Y) with Bayesian RidgeTarget: **earnings = price_mean * production_mass_t**Holdout: last **2 years**.

In [ ]:
import sysfrom pathlib import Pathimport matplotlib.pyplot as pltimport pandas as pdPROJECT_ROOT = Path("..").resolve()sys.path.append(str(PROJECT_ROOT))from src.models.integration_model import (    build_integration_dataset,    fit_bayesian_ridge,    summarize_metrics,)

## 1) Build integration dataset

In [ ]:
crop = "wheat"data_dir = PROJECT_ROOT / "data"integration_df = build_integration_dataset(crop=crop, data_dir=data_dir)integration_df.head()

## 2) Baseline (full features)

In [ ]:
result_full = fit_bayesian_ridge(crop=crop, test_size=2, data_dir=data_dir)metrics_full = summarize_metrics([result_full])metrics_full

## 3) Minimal features (price_mean + production_mass_t)

In [ ]:
result_min = fit_bayesian_ridge(    crop=crop,    test_size=2,    data_dir=data_dir,    feature_set="minimal",)metrics_min = summarize_metrics([result_min])metrics_min

## 4) Log target (full features)

In [ ]:
result_log = fit_bayesian_ridge(    crop=crop,    test_size=2,    data_dir=data_dir,    feature_set="full",    target_transform="log1p",)metrics_log = summarize_metrics([result_log])metrics_log

## 5) Compare metrics

In [ ]:
summary = pd.concat(    [        metrics_full.assign(variant="full"),        metrics_min.assign(variant="minimal"),        metrics_log.assign(variant="full_log1p"),    ],    ignore_index=True,)summary

## 6) Plot test predictions (best variant)

In [ ]:
best_row = summary.sort_values("rmse").iloc[0]variant = best_row["variant"]variant_map = {    "full": result_full,    "minimal": result_min,    "full_log1p": result_log,}best_result = variant_map[variant]plt.figure(figsize=(6, 4))plt.plot(best_result.test_index, best_result.y_true, marker="o", label="actual")plt.plot(best_result.test_index, best_result.y_pred, marker="o", label=f"pred ({variant})")plt.title(f"Earnings Forecast (Holdout) - {crop.title()}")plt.xlabel("Year")plt.ylabel("Earnings")plt.grid(True, alpha=0.3)plt.legend()plt.tight_layout()plt.show()

## 7) Multi-crop comparison (optional)

In [ ]:
rows = []for crop_name in ["wheat", "barley", "maize"]:    res_full = fit_bayesian_ridge(crop=crop_name, test_size=2, data_dir=data_dir)    res_min = fit_bayesian_ridge(        crop=crop_name, test_size=2, data_dir=data_dir, feature_set="minimal"    )    res_log = fit_bayesian_ridge(        crop=crop_name, test_size=2, data_dir=data_dir, target_transform="log1p"    )    df = pd.concat(        [            summarize_metrics([res_full]).assign(variant="full"),            summarize_metrics([res_min]).assign(variant="minimal"),            summarize_metrics([res_log]).assign(variant="full_log1p"),        ],        ignore_index=True,    )    df.insert(0, "crop", crop_name)    rows.append(df)summary_all = pd.concat(rows, ignore_index=True)summary_all